In [119]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint
from datetime import datetime, timedelta, timezone
import pytz
import scipy.stats
from dateutil.relativedelta import relativedelta

import warnings

# Suppress the specific warnings
warnings.filterwarnings("ignore")

today = datetime.utcnow()

from api_keys import espn_link

In [120]:
homeOnlySked = pd.read_csv('data/sked.csv', index_col=False)
homeOnlySked

completeSked = pd.read_csv('data/sked_full.csv', index_col=False)
completeSked

team_names = pd.read_csv('data/team_names.csv', index_col=False)
team_names

file_name = f"data/allG_df_fp-{today.strftime('%Y-%m-%d')}.csv"
allG_df = pd.read_csv(file_name, index_col=False)
allG_df

file_name = f"data/all_df_fp-{today.strftime('%Y-%m-%d')}.csv"
all_df = pd.read_csv(file_name, index_col=False)
all_df

team_names_only = team_names[['abbreviation', 'name']]

all_df_forwards =  all_df.loc[all_df['position'] != 'D']
all_df_defense =  all_df.loc[all_df['position'] == 'D']

all_df_centers =  all_df.loc[all_df['secondaryPosition'] == 'C']
all_df_wingers =  all_df.loc[all_df['secondaryPosition'] == 'W']

file_name = f"data/summary_stats-{today.strftime('%Y-%m-%d')}.csv"
player_bios = pd.read_csv(file_name, index_col=False)

# file_name = f"data/rankings_worksheet_{(today-timedelta(days=6)).strftime('%Y-%m-%d')}.xlsx"
# rankings = pd.read_excel(file_name)

In [121]:
startDate = '2023-11-20'
endDate = '2023-11-22'

In [122]:
lookahead_sked = homeOnlySked.loc[(homeOnlySked['gameDate'] >= startDate) & (homeOnlySked['gameDate'] <= endDate)]
lookahead_sked.sort_values('gameTime')

,gameID,gameDate,gameTime,awayTeam,homeTeam,gameDT,winningGoalie
253,2023020272,2023-11-20,Monday 07:00 PM,EDM,FLA,2023-11-21 00:00:00,0
542,2023020273,2023-11-20,Monday 07:00 PM,BOS,TBL,2023-11-21 00:00:00,0
212,2023020274,2023-11-20,Monday 08:00 PM,NYR,DAL,2023-11-21 01:00:00,0
1115,2023020275,2023-11-20,Monday 08:00 PM,COL,NSH,2023-11-21 01:00:00,0
703,2023020276,2023-11-20,Monday 09:00 PM,LAK,ARI,2023-11-21 02:00:00,0
131,2023020278,2023-11-20,Monday 10:00 PM,SJS,VAN,2023-11-21 03:00:00,0
787,2023020277,2023-11-20,Monday 10:00 PM,CGY,SEA,2023-11-21 03:00:00,0
665,2023020283,2023-11-22,Wednesday 07:00 PM,NYR,PIT,2023-11-23 00:00:00,0
543,2023020284,2023-11-22,Wednesday 07:00 PM,WPG,TBL,2023-11-23 00:00:00,0
625,2023020281,2023-11-22,Wednesday 07:00 PM,NJD,DET,2023-11-23 00:00:00,0


In [123]:
teams_data = {}

for index, row in lookahead_sked.iterrows():

    home_team = row['homeTeam']
    away_team = row['awayTeam']

    if home_team not in teams_data:
        teams_data[home_team] = {'played': 0, 'home': 0, 'away': 0}
    if away_team not in teams_data:
        teams_data[away_team] = {'played': 0, 'home': 0, 'away': 0}

    teams_data[home_team]['played'] += 1
    teams_data[away_team]['played'] += 1
    teams_data[home_team]['home'] += 1
    teams_data[away_team]['away'] += 1

# Create a DataFrame
teams_df = pd.DataFrame.from_dict(teams_data, orient='index')
teams_df = teams_df.sort_values(by=['played', 'home'], ascending=False)

for index, row in teams_df.iterrows():
    abbrev = index
    full_name = team_names.loc[team_names['abbreviation'] == abbrev]['name'].iloc[0]
    teams_df.at[index, 'team_name'] = full_name
    
teams_df

,played,home,away,team_name
DAL,2,2,0,Dallas Stars
FLA,2,2,0,Florida Panthers
TBL,2,2,0,Tampa Bay Lightning
ARI,2,2,0,Arizona Coyotes
SEA,2,2,0,Seattle Kraken
NSH,2,2,0,Nashville Predators
VAN,2,1,1,Vancouver Canucks
COL,2,1,1,Colorado Avalanche
SJS,2,0,2,San Jose Sharks
NYR,2,0,2,New York Rangers


In [124]:
team_check = teams_df['team_name'].to_list()

nhl_teams = [
    'New Jersey Devils', 'New York Islanders', 'New York Rangers',
    'Philadelphia Flyers', 'Pittsburgh Penguins', 'Boston Bruins',
    'Buffalo Sabres', 'Montréal Canadiens', 'Ottawa Senators',
    'Toronto Maple Leafs', 'Carolina Hurricanes', 'Florida Panthers',
    'Tampa Bay Lightning', 'Washington Capitals', 'Chicago Blackhawks',
    'Detroit Red Wings', 'Nashville Predators', 'St. Louis Blues',
    'Calgary Flames', 'Colorado Avalanche', 'Edmonton Oilers',
    'Vancouver Canucks', 'Anaheim Ducks', 'Dallas Stars',
    'Los Angeles Kings', 'San Jose Sharks', 'Columbus Blue Jackets',
    'Minnesota Wild', 'Winnipeg Jets', 'Arizona Coyotes',
    'Vegas Golden Knights', 'Seattle Kraken'
]

# # teams_df.index.values

for team in nhl_teams:

    if team not in team_check:
        print(team)

Ottawa Senators
Toronto Maple Leafs
Minnesota Wild


In [125]:
all_df_even = all_df.copy()
all_df_even['evenGoals'] = all_df_even['goals'] - all_df_even['powerPlayGoals'] - all_df_even['shorthandedGoals']
all_df_even['powerPlayAssists'] = all_df_even['powerPlayPoints'] - all_df_even['powerPlayGoals']
all_df_even['shorthandedAssists'] = all_df_even['shPoints'] - all_df_even['shorthandedGoals']
all_df_even['evenAssists'] = all_df_even['assists'] - all_df_even['powerPlayAssists'] - all_df_even['shorthandedAssists']
all_df_even['evenPoints'] = all_df_even['assists'] + all_df_even['goals']
all_df_even['evenToi'] = all_df_even['toi'] - all_df_even['powerPlayToi'] - all_df_even['shorthandedToi']

all_df_even

,playerId,sweaterNumber,name,position,goals,assists,points,plusMinus,pim,hits,...,gameId,gamesPlayed,fantasyPoints,specialTeams,evenGoals,powerPlayAssists,shorthandedAssists,evenAssists,evenPoints,evenToi
0,8478178,43,D. Raddysh,D,0,0,0,1,0,1,...,2023020001,1,0.9,0,0,0,0,0,0,1060
1,8475177,44,C. de Haan,D,0,0,0,0,0,2,...,2023020001,1,0.7,0,0,0,0,0,0,649
2,8480246,48,N. Perbix,D,0,0,0,-1,2,0,...,2023020001,1,0.6,0,0,0,0,0,0,702
3,8475167,77,V. Hedman,D,0,1,1,-1,0,0,...,2023020001,1,1.9,0,0,0,0,1,1,981
4,8478416,81,E. Cernak,D,0,0,0,0,0,6,...,2023020001,1,1.3,0,0,0,0,0,0,884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9745,8477953,42,K. Kapanen,R,0,0,0,-1,2,0,...,2023020271,1,0.0,0,0,0,0,0,0,761
9746,8481543,59,N. Alexandrov,C,0,0,0,1,0,2,...,2023020271,1,0.7,0,0,0,0,0,0,332
9747,8482089,63,J. Neighbours,L,1,0,1,1,0,0,...,2023020271,1,2.6,0,1,0,0,0,1,530
9748,8476897,70,O. Sundqvist,C,0,1,1,1,0,0,...,2023020271,1,1.0,0,0,0,0,1,1,550


In [126]:
def opposition_stats(df):
    
    opponent_stats = df.groupby('opponent').agg({
        'toi': 'sum',
        'assists': 'sum',
        'goals': 'sum',
        'shots': 'sum',
        'hits': 'sum',
        'blockedShots': 'sum',
    #     'fantasyPoints': ['sum', ('mean', lambda x: round(x.mean(), 2))],
        'fantasyPoints': 'sum',
        'gameDate': 'nunique',
        'powerPlayPoints': 'sum'
        # Add more columns as needed
    })

    opponent_stats['FPAPG'] = round(opponent_stats['fantasyPoints'] / opponent_stats['gameDate'], 2)
    opponent_stats['FPAPG_Rank'] = opponent_stats['FPAPG'].rank(ascending=False, method='min').astype(int)

    # opponent_stats['BPG'] = round(opponent_stats['blocked'] / opponent_stats['date'], 2)
    # opponent_stats['BPG_Rank'] = opponent_stats['BPG'].rank(ascending=True, method='min').astype(int)

    opponent_stats = opponent_stats.rename_axis('abbreviation')

    opponent_stats = pd.merge(opponent_stats, team_names_only, on='abbreviation')

    # opponent_stats['powerPlayPoints'] = opponent_stats['powerPlayGoals'] + opponent_stats['powerPlayAssists']

    # opponent_stats = opponent_stats.drop(columns=['powerPlayGoals', 'powerPlayAssists'])

    return opponent_stats

In [148]:
opp_all = opposition_stats(all_df)
opp_forward = opposition_stats(all_df_forwards)
opp_defense = opposition_stats(all_df_defense)
opp_centers = opposition_stats(all_df_centers)
opp_wingers = opposition_stats(all_df_wingers)

opp_even = opposition_stats(all_df_even)

opp_centers

,abbreviation,toi,assists,goals,shots,hits,blockedShots,fantasyPoints,gameDate,powerPlayPoints,FPAPG,FPAPG_Rank,name
0,ANA,102129,37,19,205,74,48,135.9,18,18,7.55,21,Anaheim Ducks
1,ARI,93229,31,19,211,102,45,129.3,17,13,7.61,19,Arizona Coyotes
2,BOS,94597,22,16,183,90,52,110.3,16,5,6.89,29,Boston Bruins
3,BUF,89970,34,19,178,108,38,124.1,18,8,6.89,29,Buffalo Sabres
4,CAR,85289,35,18,137,61,86,139.3,17,9,8.19,12,Carolina Hurricanes
5,CBJ,101739,39,27,217,47,67,156.9,19,5,8.26,9,Columbus Blue Jackets
6,CGY,90600,35,16,172,69,46,119.1,17,8,7.01,27,Calgary Flames
7,CHI,95378,39,28,221,116,50,162.2,16,15,10.14,2,Chicago Blackhawks
8,COL,82993,27,22,143,60,55,125.8,16,11,7.86,17,Colorado Avalanche
9,DAL,87179,23,16,182,99,64,117.1,16,3,7.32,24,Dallas Stars


In [128]:
# g_last_ten_df_team = pd.DataFrame()

# for index, row in team_names_only.iterrows():
#     temp_all = allG_df.loc[allG_df['team'] == row['abbreviation']].sort_values('gameDate', ascending=False)
#     five_dates = temp_all['gameDate'].unique()[:10]
#     temp_filtered = temp_all[temp_all['gameDate'].isin(five_dates)]
#     g_last_ten_df_team = pd.concat([g_last_ten_df_team, temp_filtered])
    
# g_last_ten_df_team

# gopponent_stats_10 = g_last_ten_df_team.groupby('opponent').agg({
#     'toi': 'sum',
#     'goalsAgainst': 'sum',
#     'shots': 'sum',
#     'fantasyPoints': 'sum',
#     'gameDate': 'nunique',
#     'evSaves': 'sum',
#     'evShots': 'sum',
#     'shutout': 'sum',
#     'ppSaves': 'sum',
#     'ppShots': 'sum'
# })

# gopponent_stats_10['FPAPG'] = round(gopponent_stats_10['fantasyPoints'] / gopponent_stats_10['gameDate'], 2)
# gopponent_stats_10['FPAPG_Rank'] = gopponent_stats_10['FPAPG'].rank(ascending=False, method='min').astype(int)

# gopponent_stats_10 = gopponent_stats_10.rename_axis('abbreviation')

# gopponent_stats_10 = pd.merge(gopponent_stats_10, team_names_only, on='abbreviation')

# gopponent_stats_10

In [129]:
gopponent_stats = allG_df.groupby('opponent').agg({
    'toi': 'sum',
    'goalsAgainst': 'sum',
    'shots': 'sum',
#     'fantasyPoints': ['sum', ('mean', lambda x: round(x.mean(), 2))],
    'fantasyPoints': 'sum',
    'gameDate': 'nunique',
    'evSaves': 'sum',
    'evShots': 'sum',
    'shutout': 'sum',
    'ppSaves': 'sum',
    'ppShots': 'sum'
    # Add more columns as needed
})

gopponent_stats['FPAPG'] = round(gopponent_stats['fantasyPoints'] / gopponent_stats['gameDate'], 2)
gopponent_stats['FPAPG_Rank'] = gopponent_stats['FPAPG'].rank(ascending=False, method='min').astype(int)

# gopponent_stats['SPG'] = round(gopponent_stats['shots'] / gopponent_stats['date'], 2)
# gopponent_stats['SPG_Rank'] = gopponent_stats['SPG'].rank(ascending=False, method='min').astype(int)

gopponent_stats = gopponent_stats.rename_axis('abbreviation')

gopponent_stats = pd.merge(gopponent_stats, team_names_only, on='abbreviation')

# gopponent_stats['powerPlayPoints'] = gopponent_stats['powerPlayGoals'] + gopponent_stats['powerPlayAssists']

# gopponent_stats = gopponent_stats.drop(columns=['powerPlayGoals', 'powerPlayAssists', 'id'])

gopponent_stats

,abbreviation,toi,goalsAgainst,shots,fantasyPoints,gameDate,evSaves,evShots,shutout,ppSaves,ppShots,FPAPG,FPAPG_Rank,name
0,ANA,64792,48,517,36.8,18,374,408,0,77,89,2.04,15,Anaheim Ducks
1,ARI,61410,54,469,16.0,17,348,384,1,60,78,0.94,23,Arizona Coyotes
2,BOS,57046,51,514,4.6,16,360,398,0,87,100,0.29,29,Boston Bruins
3,BUF,64392,47,507,42.0,18,378,418,1,72,78,2.33,12,Buffalo Sabres
4,CAR,61688,54,577,31.6,17,432,468,1,73,87,1.86,16,Carolina Hurricanes
5,CBJ,68587,49,584,76.0,19,429,472,2,94,100,4.00,2,Columbus Blue Jackets
6,CGY,61554,43,553,63.0,17,431,465,1,67,74,3.71,3,Calgary Flames
7,CHI,56894,37,419,53.4,16,319,350,2,49,55,3.34,5,Chicago Blackhawks
8,COL,57326,55,544,17.8,16,382,421,3,90,102,1.11,20,Colorado Avalanche
9,DAL,57610,52,488,9.2,16,350,387,1,69,80,0.58,27,Dallas Stars


In [130]:
player_bios.loc[player_bios['name'] == 'A. Ovechkin']

,playerId,name,team,position,toi,gamesPlayed,fantasyPoints,FPP60,FPPG,missedGames,gamesRemaining,fantasyPointsRemain,fullName,birthDate,shootsCatches,height,weight,roster_percent,default_pos,espnId
113,8471214,A. Ovechkin,WSH,L,18058,15,29.4,5.86,1.96,0,67,131.32,Alex Ovechkin,1985-09-17,R,75,238,99.362786,LW,3101


In [131]:
team_names_only

last_five_df_team = pd.DataFrame()

for index, row in team_names_only.iterrows():
    temp_all = all_df.loc[all_df['team'] == row['abbreviation']].sort_values('gameDate', ascending=False)
    five_dates = temp_all['gameDate'].unique()[:5]
    temp_filtered = temp_all[temp_all['gameDate'].isin(five_dates)]
    last_five_df_team = pd.concat([last_five_df_team, temp_filtered])
    
last_five_df_team

,playerId,sweaterNumber,name,position,goals,assists,points,plusMinus,pim,hits,...,team,opponent,secondaryPosition,tertiaryPosition,gameDate,gameTime,gameId,gamesPlayed,fantasyPoints,specialTeams
8975,8480880,94,J. Lauko,C,0,0,0,-2,0,3,...,BOS,MTL,C,F,2023-11-18 00:00:00,Saturday 07:00 PM,2023020258,1,0.5,0
8966,8478401,18,P. Zacha,C,1,0,1,1,0,0,...,BOS,MTL,C,F,2023-11-18 00:00:00,Saturday 07:00 PM,2023020258,1,2.8,0
8958,8482511,6,M. Lohrei,D,0,0,0,-1,0,1,...,BOS,MTL,D,D,2023-11-18 00:00:00,Saturday 07:00 PM,2023020258,1,2.3,0
8959,8474031,12,K. Shattenkirk,D,0,0,0,-1,0,1,...,BOS,MTL,D,D,2023-11-18 00:00:00,Saturday 07:00 PM,2023020258,1,0.5,0
8960,8478443,25,B. Carlo,D,0,1,1,3,0,1,...,BOS,MTL,D,D,2023-11-18 00:00:00,Saturday 07:00 PM,2023020258,1,1.2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3646,8479379,54,G. Smith,R,0,0,0,0,0,2,...,SJS,EDM,W,F,2023-11-09,Thursday 10:30 PM,2023020203,1,0.2,0
3648,8475798,64,M. Granlund,C,0,0,0,0,2,0,...,SJS,EDM,C,F,2023-11-09,Thursday 10:30 PM,2023020203,1,0.6,0
3649,8474884,68,M. Hoffman,C,0,0,0,-1,0,0,...,SJS,EDM,C,F,2023-11-09,Thursday 10:30 PM,2023020203,1,1.1,0
3650,8482667,72,W. Eklund,L,0,1,1,1,2,0,...,SJS,EDM,W,F,2023-11-09,Thursday 10:30 PM,2023020203,1,1.6,0


In [132]:
def summary_statistics(df):
    temp_build = df.groupby('playerId').agg({
        'name': 'first',
        'team': 'last',
        'position': 'first',
        'toi': 'sum',
        'gamesPlayed': 'sum',
        'goals': 'sum',
        'assists': 'sum',
        'specialTeams': 'sum',
        'shots': 'sum',
        'hits': 'sum',
        'blockedShots': 'sum',
        'powerPlayToi': 'sum',
        'shorthandedToi': 'sum',
        'plusMinus': 'sum',
        'fantasyPoints': 'sum',
        'secondaryPosition': 'first',
        'tertiaryPosition': 'first'
    }).reset_index()



    temp_build['FPP60'] = (temp_build['fantasyPoints'] / temp_build['toi'] * 3600).round(2)
    temp_build['FPPG'] = (temp_build['fantasyPoints'] / temp_build['gamesPlayed']).round(2)

    temp_build = temp_build.sort_values(by='FPPG', ascending=False)
    
    return temp_build

In [133]:


top_players_by_team = summary_statistics(last_five_df_team).groupby('team').apply(lambda x: x.nlargest(5, 'fantasyPoints'))
top_players_by_team.reset_index(level='team', drop=True, inplace=True)
top_players_by_team

for index, row in top_players_by_team.iterrows():
    top_players_by_team.at[index, 'available'] = 100 - round(player_bios.loc[player_bios['playerId'] == row['playerId']]['roster_percent'].iloc[0], 2)

top_players_by_team

,playerId,name,team,position,toi,gamesPlayed,goals,assists,specialTeams,shots,...,blockedShots,powerPlayToi,shorthandedToi,plusMinus,fantasyPoints,secondaryPosition,tertiaryPosition,FPP60,FPPG,available
93,8475462,R. Gudas,ANA,D,5800,5,2,2,0,6,...,11,30,1061,4,13.9,D,D,8.63,2.78,51.15
323,8478366,F. Vatrano,ANA,R,5891,5,3,2,2,20,...,3,797,738,1,13.1,W,F,8.01,2.62,10.62
538,8480950,I. Lyubushkin,ANA,D,5564,5,0,1,0,2,...,17,1,1020,-5,10.8,D,D,6.99,2.16,96.34
643,8482745,M. McTavish,ANA,C,5187,5,2,1,1,12,...,5,825,7,0,9.4,C,F,6.52,1.88,26.06
109,8475764,C. Fowler,ANA,D,6888,5,1,2,2,7,...,6,856,1111,-5,8.9,D,D,4.65,1.78,42.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,8471214,A. Ovechkin,WSH,L,5940,5,3,1,0,15,...,4,1374,0,3,12.1,W,F,7.33,2.42,0.64
530,8480873,R. Sandin,WSH,D,7614,5,0,3,0,6,...,16,433,269,6,11.9,D,D,5.63,2.38,80.16
588,8481656,A. Protas,WSH,C,3125,5,2,4,1,8,...,3,12,42,5,10.9,C,F,12.56,2.18,99.92
402,8479359,B. Malenstyn,WSH,L,4503,5,2,1,0,12,...,7,11,767,2,10.9,W,F,8.71,2.18,99.85


In [134]:

worst_five = summary_statistics(last_five_df_team).groupby('team').apply(lambda x: x.nsmallest(10, 'fantasyPoints'))
worst_five.reset_index(level='team', drop=True, inplace=True)

for index, row in worst_five.iterrows():
    worst_five.at[index, 'available'] = 100 - round(player_bios.loc[player_bios['playerId'] == row['playerId']]['roster_percent'].iloc[0], 2)
    
worst_five = worst_five.loc[worst_five['available'] <= 25]    

worst_five

,playerId,name,team,position,toi,gamesPlayed,goals,assists,specialTeams,shots,...,blockedShots,powerPlayToi,shorthandedToi,plusMinus,fantasyPoints,secondaryPosition,tertiaryPosition,FPP60,FPPG,available
282,8477949,A. Tuch,BUF,R,2507,2,1,0,0,4,...,0,324,438,1,2.5,W,F,3.59,1.25,13.92
424,8479420,T. Thompson,BUF,C,2744,3,0,1,0,3,...,3,564,374,-2,2.8,C,F,3.67,0.93,3.06
517,8480830,A. Svechnikov,CAR,R,5123,5,0,2,0,11,...,1,569,0,0,4.9,W,F,3.44,0.98,13.30
279,8477946,D. Larkin,DET,C,5963,5,0,0,0,15,...,2,1277,343,-3,2.8,C,F,1.69,0.56,1.23
294,8477986,B. Montour,FLA,D,1442,1,0,0,0,1,...,0,261,0,1,0.1,D,D,0.25,0.10,11.19
564,8481540,C. Caufield,MTL,R,5152,5,0,2,1,16,...,0,1045,0,-3,4.3,W,F,3.00,0.86,2.45
573,8481559,J. Hughes,NJD,C,1360,1,1,1,0,8,...,0,58,0,-1,3.8,C,F,10.06,3.80,0.47
153,8476459,M. Zibanejad,NYR,C,5813,5,0,1,1,12,...,2,722,486,4,4.0,C,F,2.48,0.80,3.04
473,8480064,J. Norris,OTT,C,5554,5,1,1,0,10,...,3,972,5,1,6.1,C,F,3.95,1.22,22.99
110,8475765,V. Tarasenko,OTT,R,4977,5,0,3,1,7,...,3,610,1,3,6.3,W,F,4.56,1.26,15.66


In [135]:
team_names_only


last_five_df = pd.DataFrame()

for index, row in team_names_only.iterrows():
    temp_all = allG_df.loc[allG_df['team'] == row['abbreviation']].sort_values('gameDate', ascending=False)
    five_dates = temp_all['gameDate'].unique()[:5]
    temp_filtered = temp_all[temp_all['gameDate'].isin(five_dates)]
    last_five_df = pd.concat([last_five_df, temp_filtered])
    
last_five_df

,playerId,sweaterNumber,name,position,pim,goalsAgainst,toi,team,opponent,gameDate,...,win,saves,shots,evSaves,evShots,ppSaves,ppShots,gamesPlayed,fantasyPoints,decisionType
529,8480280,1,J. Swayman,G,0,2,3600,BOS,MTL,2023-11-18 00:00:00,...,4,20,22,15,17,4,4,1,4.0,W
500,8476999,35,L. Ullmark,G,0,2,3561,BOS,BUF,2023-11-14 00:00:00,...,4,32,34,27,29,4,4,1,6.4,W
467,8480280,1,J. Swayman,G,0,3,3733,BOS,MTL,2023-11-11,...,1,24,27,20,22,3,4,1,-0.2,OTL
206,8476999,35,L. Ullmark,G,0,2,3600,BOS,NYI,2023-11-09,...,4,27,29,24,24,2,3,1,5.4,W
432,8480280,1,J. Swayman,G,0,2,3571,BOS,DAL,2023-11-06,...,4,35,37,30,32,4,4,1,7.0,W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521,8478039,36,K. Kahkonen,G,0,1,3584,SJS,STL,2023-11-16 00:00:00,...,4,44,45,34,35,9,9,1,10.8,W
493,8478406,29,M. Blackwood,G,0,4,3434,SJS,FLA,2023-11-14 00:00:00,...,0,26,30,21,23,4,6,1,-2.8,L
481,8478406,29,M. Blackwood,G,0,4,3600,SJS,ANA,2023-11-12,...,0,40,44,27,29,12,14,1,0.0,L
464,8478039,36,K. Kahkonen,G,0,5,3600,SJS,VGK,2023-11-10,...,0,34,39,33,37,0,0,1,-3.2,L


In [136]:
summary_stats_last5G = last_five_df.groupby('playerId').agg({
    'name': 'first',
    'team': 'last',
    'position': 'first',
    'toi': 'sum',
    'gamesPlayed': 'sum',
    'saves': 'sum',
    'shots': 'sum',
    'start': 'sum',
    'shutout': 'sum',
    'fantasyPoints': 'sum'
}).reset_index()

summary_stats_last5G.sort_values('fantasyPoints', ascending=False).head(25)
summary_stats_last5G

for index, row in summary_stats_last5G.iterrows():
    summary_stats_last5G.at[index, 'available'] = 100 - round(player_bios.loc[player_bios['playerId'] == row['playerId']]['roster_percent'].iloc[0], 2)

summary_stats_last5G['FPP60'] = round(summary_stats_last5G['fantasyPoints'] / summary_stats_last5G['toi'] *3600, 2)
summary_stats_last5G

,playerId,name,team,position,toi,gamesPlayed,saves,shots,start,shutout,fantasyPoints,available,FPP60
0,8470594,M. Fleury,MIN,G,10853,3,74,89,3,0,-14.2,84.21,-4.71
1,8471734,J. Quick,NYR,G,11276,3,85,95,3,0,6.0,81.27,1.92
2,8473503,J. Reimer,DET,G,7698,2,53,61,2,0,-3.4,89.44,-1.59
3,8473575,S. Varlamov,NYI,G,7277,2,53,59,2,0,-0.4,74.94,-0.20
4,8474593,J. Markstrom,CGY,G,11021,3,83,90,3,0,11.6,49.43,3.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,8481611,P. Kochetkov,CAR,G,7079,2,43,46,2,1,9.6,70.38,4.88
65,8481692,D. Wolf,CGY,G,3585,1,34,38,1,0,-1.2,98.24,-1.21
66,8482221,D. Levi,BUF,G,5707,2,46,53,2,0,-0.8,76.60,-0.50
67,8482411,H. Shepard,WSH,G,3600,1,36,37,1,0,9.2,99.93,9.20


In [137]:
top_players_by_team

,playerId,name,team,position,toi,gamesPlayed,goals,assists,specialTeams,shots,...,blockedShots,powerPlayToi,shorthandedToi,plusMinus,fantasyPoints,secondaryPosition,tertiaryPosition,FPP60,FPPG,available
93,8475462,R. Gudas,ANA,D,5800,5,2,2,0,6,...,11,30,1061,4,13.9,D,D,8.63,2.78,51.15
323,8478366,F. Vatrano,ANA,R,5891,5,3,2,2,20,...,3,797,738,1,13.1,W,F,8.01,2.62,10.62
538,8480950,I. Lyubushkin,ANA,D,5564,5,0,1,0,2,...,17,1,1020,-5,10.8,D,D,6.99,2.16,96.34
643,8482745,M. McTavish,ANA,C,5187,5,2,1,1,12,...,5,825,7,0,9.4,C,F,6.52,1.88,26.06
109,8475764,C. Fowler,ANA,D,6888,5,1,2,2,7,...,6,856,1111,-5,8.9,D,D,4.65,1.78,42.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,8471214,A. Ovechkin,WSH,L,5940,5,3,1,0,15,...,4,1374,0,3,12.1,W,F,7.33,2.42,0.64
530,8480873,R. Sandin,WSH,D,7614,5,0,3,0,6,...,16,433,269,6,11.9,D,D,5.63,2.38,80.16
588,8481656,A. Protas,WSH,C,3125,5,2,4,1,8,...,3,12,42,5,10.9,C,F,12.56,2.18,99.92
402,8479359,B. Malenstyn,WSH,L,4503,5,2,1,0,12,...,7,11,767,2,10.9,W,F,8.71,2.18,99.85


In [138]:
df = lookahead_sked.copy()

df.rename(columns={'homeTeam': 'focusTeam', 'gameDate': 'date'}, inplace=True)
df_temp = df[['gameID', 'date', 'awayTeam', 'focusTeam', 'gameTime', 'gameDT']]
df_temp.rename(columns={'focusTeam': 'opponent', 'date': 'date', 'awayTeam': 'focusTeam'}, inplace=True)
df.rename(columns={'awayTeam': 'opponent'}, inplace=True)
df['H/A'] = 'H'
df_temp['H/A'] = 'A'
df = pd.concat([df, df_temp])
df.drop(columns=['winningGoalie'], inplace=True)

sked_stats = df
sked_stats.reset_index(inplace=True)
sked_stats = sked_stats.sort_values('gameTime')

sked_stats = sked_stats[['gameID', 'date', 'gameTime', 'focusTeam', 'H/A', 'opponent']]
# sked_stats['opponent'] = sked_stats.groupby('focusTeam')['opponent'].sum()
sked_stats


for index, row in sked_stats.iterrows():
    d = opp_defense.loc[opp_defense['abbreviation'] == row['opponent']]['FPAPG'].iloc[0]
#     drank = D.loc[D.index == row['focus']]['FPAPG_Rank'][0]
    c = opp_centers.loc[opp_centers['abbreviation'] == row['opponent']]['FPAPG'].iloc[0]
#     crank = C.loc[C.index == row['focus']]['FPAPG_Rank'][0]
    w = opp_wingers.loc[opp_wingers['abbreviation'] == row['opponent']]['FPAPG'].iloc[0]
#     wrank = W.loc[W.index == row['focus']]['FPAPG_Rank'][0]
    g = gopponent_stats.loc[gopponent_stats['abbreviation'] == row['opponent']]['FPAPG'].iloc[0]
    
    sked_stats.at[index, 'D_FPAPG'] = d
#     sked_stats.at[index, 'D_RANK'] = drank
    sked_stats.at[index, 'C_FPAPG'] = c
#     sked_stats.at[index, 'C_RANK'] = crank
    sked_stats.at[index, 'W_FPAPG'] = w
#     sked_stats.at[index, 'W_RANK'] = wrank
    sked_stats.at[index, 'G_FPAPG'] = g
    
sked_summary = sked_stats.groupby('focusTeam').sum().sort_values('D_FPAPG', ascending=False)
sked_summary['D_rank'] = sked_summary['D_FPAPG'].rank(ascending=False, method='first').astype(int)
sked_summary['C_rank'] = sked_summary['C_FPAPG'].rank(ascending=False, method='first').astype(int)
sked_summary['W_rank'] = sked_summary['W_FPAPG'].rank(ascending=False, method='first').astype(int)
sked_summary['G_rank'] = sked_summary['G_FPAPG'].rank(ascending=False, method='first').astype(int)
sked_summary

sked_summary.sort_values('D_rank', ascending=True, inplace=True)
sked_summary

d_adds = pd.DataFrame()
for index, row in sked_summary.iterrows():
    temp = top_players_by_team.loc[(top_players_by_team['team'] == index) & (top_players_by_team['position'] == 'D')]
    temp = temp.loc[temp['available'] >= 25]
    temp['D_rank'] = row['D_rank'].astype(int)
    d_adds = pd.concat([d_adds, temp])
    if row['D_rank'] == 5:
        break

sked_summary.sort_values('C_rank', ascending=True, inplace=True)
c_adds = pd.DataFrame()
for index, row in sked_summary.iterrows():
    temp = top_players_by_team.loc[(top_players_by_team['team'] == index) & (top_players_by_team['secondaryPosition'] == 'C')]
    temp = temp.loc[temp['available'] >= 25]
    temp['C_rank'] = row['C_rank'].astype(int)
    c_adds = pd.concat([c_adds, temp])
    if row['C_rank'] == 5:
        break
        
sked_summary.sort_values('W_rank', ascending=True, inplace=True)
w_adds = pd.DataFrame()
for index, row in sked_summary.iterrows():
    temp = top_players_by_team.loc[(top_players_by_team['team'] == index) & (top_players_by_team['secondaryPosition'] == 'W')]
    temp = temp.loc[temp['available'] >= 25]
    temp['W_rank'] = row['W_rank'].astype(int)
    w_adds = pd.concat([w_adds, temp])
    if row['W_rank'] == 5:
        break
        
sked_summary.sort_values('G_rank', ascending=True, inplace=True)
g_adds = pd.DataFrame()
for index, row in sked_summary.iterrows():
    temp = summary_stats_last5G.loc[(summary_stats_last5G['team'] == index)]
    temp = temp.loc[temp['available'] >= 25]
    temp['G_rank'] = row['G_rank'].astype(int)
    g_adds = pd.concat([g_adds, temp])
    if row['G_rank'] == 5:
        break
        
adds = pd.concat([d_adds, c_adds, w_adds])
adds['merged_rank'] = adds['D_rank'].combine_first(adds['C_rank']).combine_first(adds['W_rank'])
adds = adds.drop(['D_rank', 'C_rank', 'W_rank'], axis=1)
sked_stats

for index, row in adds.iterrows(): 
#     print(f"<p><b>{player_bios.loc[player_bios['playerId'] == row['playerId']]['fullName'].iloc[0]}, {row['position']}, {team_names_only.loc[team_names_only['abbreviation'] == row['team']]['name'].iloc[0]} </b>({round(row['available'], 2)}% available): {round(row['fantasyPoints'], 2)} fantasy points ({row['FPPG']} FPPG)")
    print(
    f"<p><b>{player_bios.loc[player_bios['playerId'] == row['playerId']]['fullName'].iloc[0]}, "
    f"{row['position']}, {team_names_only.loc[team_names_only['abbreviation'] == row['team']]['name'].iloc[0]} "
    f"</b>({round(row['available'], 2)}% available): {round(row['fantasyPoints'], 2)} fantasy points "
    f"({row['FPPG']} FPPG) {sked_stats.groupby('focusTeam')['opponent'].sum()[row['team']]}"
)
    
for index, row in g_adds.iterrows(): 
#     print(f"<p><b>{player_bios.loc[player_bios['playerId'] == row['playerId']]['fullName'].iloc[0]}, {row['position']}, {team_names_only.loc[team_names_only['abbreviation'] == row['team']]['name'].iloc[0]} </b>({round(row['available'], 2)}% available): {round(row['fantasyPoints'], 2)} fantasy points ({row['FPPG']} FPPG)")
    print(
    f"<p><b>{player_bios.loc[player_bios['playerId'] == row['playerId']]['fullName'].iloc[0]}, "
    f"{row['position']}, {team_names_only.loc[team_names_only['abbreviation'] == row['team']]['name'].iloc[0]} "
    f"</b>({round(row['available'], 2)}% available): {round(row['fantasyPoints'], 2)} fantasy points "
    f"({row['FPP60']} FPP60) {sked_stats.groupby('focusTeam')['opponent'].sum()[row['team']]}"
)

<p><b>MacKenzie Weegar, D, Calgary Flames </b>(27.76% available): 13.0 fantasy points (2.6 FPPG) SEANSH
<p><b>Jamie Oleksiak, D, Seattle Kraken </b>(88.36% available): 11.2 fantasy points (2.24 FPPG) CGYSJS
<p><b>Nikita Okhotiuk, D, San Jose Sharks </b>(99.83% available): 12.3 fantasy points (2.46 FPPG) VANSEA
<p><b>Devon Toews, D, Colorado Avalanche </b>(30.21% available): 13.6 fantasy points (2.72 FPPG) NSHVAN
<p><b>Oliver Ekman-Larsson, D, Florida Panthers </b>(52.84% available): 9.1 fantasy points (1.82 FPPG) EDMBOS
<p><b>Tomas Hertl, C, San Jose Sharks </b>(28.73% available): 12.7 fantasy points (2.54 FPPG) VANSEA
<p><b>Mike Hoffman, C, San Jose Sharks </b>(99.87% available): 10.2 fantasy points (2.04 FPPG) VANSEA
<p><b>Luke Kunin, C, San Jose Sharks </b>(99.7% available): 9.9 fantasy points (1.98 FPPG) VANSEA
<p><b>Charlie Coyle, C, Boston Bruins </b>(82.44% available): 13.8 fantasy points (2.76 FPPG) TBLFLA
<p><b>Trent Frederic, C, Boston Bruins </b>(99.08% available): 11.2 fant

In [147]:
sked_summary

,gameID,D_FPAPG,C_FPAPG,W_FPAPG,G_FPAPG,D_rank,C_rank,W_rank,G_rank
focusTeam,,,,,,,,,
SEA,4046040569,20.24,16.46,18.79,9.09,2,7,1,1
VAN,4046040569,18.99,17.31,18.31,6.49,7,2,2,2
CGY,4046040565,20.70,17.14,15.34,5.41,1,4,7,3
NSH,4046040563,19.01,14.87,17.14,4.82,6,11,5,4
EDM,4046040551,18.18,17.14,17.48,4.19,9,5,3,5
CHI,2023020280,10.21,8.26,8.89,4.00,16,19,18,6
FLA,4046040554,19.15,16.88,14.96,3.95,5,6,8,7
CAR,2023020279,10.56,9.99,9.36,3.66,14,16,15,8
BOS,4046040555,18.51,17.18,17.15,3.49,8,3,4,9


In [140]:
lookahead_sked = lookahead_sked.sort_values('gameTime')
lookahead_sked

,gameID,gameDate,gameTime,awayTeam,homeTeam,gameDT,winningGoalie
253,2023020272,2023-11-20,Monday 07:00 PM,EDM,FLA,2023-11-21 00:00:00,0
542,2023020273,2023-11-20,Monday 07:00 PM,BOS,TBL,2023-11-21 00:00:00,0
212,2023020274,2023-11-20,Monday 08:00 PM,NYR,DAL,2023-11-21 01:00:00,0
1115,2023020275,2023-11-20,Monday 08:00 PM,COL,NSH,2023-11-21 01:00:00,0
703,2023020276,2023-11-20,Monday 09:00 PM,LAK,ARI,2023-11-21 02:00:00,0
131,2023020278,2023-11-20,Monday 10:00 PM,SJS,VAN,2023-11-21 03:00:00,0
787,2023020277,2023-11-20,Monday 10:00 PM,CGY,SEA,2023-11-21 03:00:00,0
665,2023020283,2023-11-22,Wednesday 07:00 PM,NYR,PIT,2023-11-23 00:00:00,0
543,2023020284,2023-11-22,Wednesday 07:00 PM,WPG,TBL,2023-11-23 00:00:00,0
625,2023020281,2023-11-22,Wednesday 07:00 PM,NJD,DET,2023-11-23 00:00:00,0


In [141]:
for index, row in lookahead_sked.iterrows():
    
    date_object_utc = datetime.strptime(row['gameDT'], "%Y-%m-%d %H:%M:%S").replace(tzinfo=pytz.UTC)
    date_object_et = date_object_utc.astimezone(pytz.timezone("US/Eastern"))
    formatted_time = date_object_et.strftime("%I:%M p.m.").lstrip('0').replace("PM", "p.m.")
    
    date_string_et = date_object_et.strftime("%Y-%m-%d %H:%M:%S%z")
    date_object = datetime.strptime(date_string_et, "%Y-%m-%d %H:%M:%S%z")    
    day_of_week = date_object.strftime("%A")
    
    home_abbr = row['homeTeam']
    home_top5 = top_players_by_team.loc[top_players_by_team['team'] == home_abbr]
    home_goalies = summary_stats_last5G.loc[summary_stats_last5G['team'] == home_abbr]

    away_abbr = row['awayTeam']
    away_top5 = top_players_by_team.loc[top_players_by_team['team'] == away_abbr]
    away_goalies = summary_stats_last5G.loc[summary_stats_last5G['team'] == away_abbr]
    
    home_team = team_names_only.loc[team_names_only['abbreviation'] == home_abbr]['name'].iloc[0]
    away_team = team_names_only.loc[team_names_only['abbreviation'] == away_abbr]['name'].iloc[0]
        
    print(f"\033[1m{away_team} at {home_team}\033[0m")
    print(f"{formatted_time}")
    print()
    print(f"\033[1mForwards against {home_abbr} score\033[0m: {opp_forward.loc[opp_forward['abbreviation'] == home_abbr, 'FPAPG'].values[0]} FPAPG, Rank: {opp_forward.loc[opp_forward['abbreviation'] == home_abbr, 'FPAPG_Rank'].values[0]}")
    print(f"\033[1mDefence against {home_abbr} score\033[0m: {opp_defense.loc[opp_defense['abbreviation'] == home_abbr, 'FPAPG'].values[0]} FPAPG, Rank: {opp_defense.loc[opp_defense['abbreviation'] == home_abbr, 'FPAPG_Rank'].values[0]}")
    print(f"\033[1mTop {away_abbr} Players Fantasy Points Last 5 \033[0m")
    for index, row in away_top5.iterrows():
        print(f"<p><b>{player_bios.loc[player_bios['playerId'] == row['playerId']]['fullName'].iloc[0]}, {row['position']}, {away_team} </b>({round(row['available'], 2)}% available): {round(row['fantasyPoints'], 2)} fantasy points ({row['FPPG']} FPPG)")
    print()
    print(f"\033[1mGoalies against {home_abbr} score\033[0m: {gopponent_stats.loc[gopponent_stats['name'] == home_team, 'FPAPG'].values[0]} FPAPG, Rank: {gopponent_stats.loc[gopponent_stats['name'] == home_team, 'FPAPG_Rank'].values[0]}")
    for index, row in away_goalies.iterrows():
        print(f"{player_bios.loc[player_bios['playerId'] == row['playerId']]['fullName'].iloc[0]} ({round(row['available'], 2)}% available) last five: {row['start']} starts, FPP60: {row['FPP60']}, FP: {round(row['fantasyPoints'], 2)}")
    print()
    print(f"\033[1mForwards against {away_abbr} score\033[0m: {opp_forward.loc[opp_forward['abbreviation'] == away_abbr, 'FPAPG'].values[0]} FPAPG, Rank: {opp_forward.loc[opp_forward['abbreviation'] == away_abbr, 'FPAPG_Rank'].values[0]}")
    print(f"\033[1mDefence against {away_abbr} score\033[0m: {opp_defense.loc[opp_defense['abbreviation'] == away_abbr, 'FPAPG'].values[0]} FPAPG, Rank: {opp_defense.loc[opp_defense['abbreviation'] == away_abbr, 'FPAPG_Rank'].values[0]}")
    print(f"\033[1mTop {home_abbr} Players Fantasy Points Last 5 \033[0m")
    for index, row in home_top5.iterrows():
        print(f"<p><b>{player_bios.loc[player_bios['playerId'] == row['playerId']]['fullName'].iloc[0]}, {row['position']}, {home_team}</b>({round(row['available'], 2)}% available): {round(row['fantasyPoints'], 2)} fantasy points ({row['FPPG']} FPPG)")

    print()
    print(f"\033[1mGoalies against {away_abbr} score\033[0m: {gopponent_stats.loc[gopponent_stats['name'] == away_team, 'FPAPG'].values[0]} FPAPG, Rank: {gopponent_stats.loc[gopponent_stats['name'] == away_team, 'FPAPG_Rank'].values[0]}")
    for index, row in home_goalies.iterrows():
        print(f"{player_bios.loc[player_bios['playerId'] == row['playerId']]['fullName'].iloc[0]} ({round(row['available'], 2)}% available) last five: {row['start']} starts, FPP60: {row['FPP60']}, FP: {round(row['fantasyPoints'], 2)}")
    print()
    print('-'*25)
    print()

Edmonton Oilers at Florida Panthers
7:00 p.m.

Forwards against FLA score: 16.89 FPAPG, Rank: 12
Defence against FLA score: 9.32 FPAPG, Rank: 14
Top EDM Players Fantasy Points Last 5 
<p><b>Evander Kane, L, Edmonton Oilers </b>(34.26% available): 14.7 fantasy points (2.94 FPPG)
<p><b>Evan Bouchard, D, Edmonton Oilers </b>(5.33% available): 14.1 fantasy points (2.82 FPPG)
<p><b>Zach Hyman, L, Edmonton Oilers </b>(8.9% available): 13.9 fantasy points (2.78 FPPG)
<p><b>Leon Draisaitl, C, Edmonton Oilers </b>(0.09% available): 11.9 fantasy points (2.38 FPPG)
<p><b>Darnell Nurse, D, Edmonton Oilers </b>(7.89% available): 11.7 fantasy points (2.34 FPPG)

Goalies against FLA score: 2.33 FPAPG, Rank: 12
Stuart Skinner (51.62% available) last five: 5 starts, FPP60: 1.77, FP: 8.8

Forwards against EDM score: 19.35 FPAPG, Rank: 2
Defence against EDM score: 10.56 FPAPG, Rank: 1
Top FLA Players Fantasy Points Last 5 
<p><b>Sam Reinhart, C, Florida Panthers</b>(1.63% available): 19.3 fantasy points 

In [146]:
temp_team_df = pd.DataFrame()

for index, row in team_names_only.iterrows():
    temp_all = all_df.loc[all_df['team'] == row['abbreviation']].sort_values('gameDate', ascending=False)
    five_dates = temp_all['gameDate'].unique()[:6]
    temp_filtered = temp_all[temp_all['gameDate'].isin(five_dates)]
    temp_team_df = pd.concat([temp_team_df, temp_filtered])
    
temp_team_df = temp_team_df.loc[(temp_team_df['team'] == 'STL') & (temp_team_df['position'] == 'D')]
temp_team_df = summary_statistics(temp_team_df)
# temp_team_df['powerPlayToi'] = f"{(temp_team_df['powerPlayToi'] // 60)}:{(temp_team_df['powerPlayToi'] % 60)}"
temp_team_df

,playerId,name,team,position,toi,gamesPlayed,goals,assists,specialTeams,shots,hits,blockedShots,powerPlayToi,shorthandedToi,plusMinus,fantasyPoints,secondaryPosition,tertiaryPosition,FPP60,FPPG
3,8475753,J. Faulk,STL,D,8244,6,0,5,0,13,7,17,1014,469,4,15.5,D,D,6.77,2.58
4,8476792,T. Krug,STL,D,7962,6,1,4,0,21,5,12,1311,194,2,14.6,D,D,6.60,2.43
5,8476892,C. Parayko,STL,D,8067,6,1,1,1,10,7,12,42,705,4,11.2,D,D,5.00,1.87
6,8481059,S. Perunovich,STL,D,5375,6,0,1,1,3,2,10,1103,0,-1,7.0,D,D,4.69,1.17
2,8475181,N. Leddy,STL,D,7709,6,0,1,0,9,0,6,17,605,3,4.9,D,D,2.29,0.82
0,8474145,R. Bortuzzo,STL,D,779,1,0,0,0,0,2,1,0,104,1,0.7,D,D,3.23,0.70
1,8474618,M. Scandella,STL,D,3774,5,0,0,0,2,4,5,11,434,-2,3.1,D,D,2.96,0.62


In [154]:
temp_player = all_df.loc[all_df['name'] == 'A. Svechnikov']
temp_player = temp_player.sort_values('gameDate', ascending=True).head(10)
summary_statistics(temp_player)

,playerId,name,team,position,toi,gamesPlayed,goals,assists,specialTeams,shots,hits,blockedShots,powerPlayToi,shorthandedToi,plusMinus,fantasyPoints,secondaryPosition,tertiaryPosition,FPP60,FPPG
0,8480830,A. Svechnikov,CAR,R,8840,9,0,5,1,19,21,2,827,4,-1,10.5,W,F,4.28,1.17


In [158]:
player_bios.sort_values('fantasyPoints', ascending=False, inplace=True)
player_bios['rank'] = player_bios['fantasyPoints'].rank(ascending=False, method='first')
player_bios.loc[player_bios['team'] == 'NYR']

,playerId,name,team,position,toi,gamesPlayed,fantasyPoints,FPP60,FPPG,missedGames,...,fantasyPointsRemain,fullName,birthDate,shootsCatches,height,weight,roster_percent,default_pos,espnId,rank
6,8478550,A. Panarin,NYR,L,17703,15,49.0,9.96,3.27,0,...,219.09,Artemi Panarin,1991-10-30,R,72,175,97.672848,LW,3891952,16.0
23,8476885,J. Trouba,NYR,D,20051,15,39.6,7.11,2.64,0,...,176.88,Jacob Trouba,1994-02-26,R,75,209,97.336505,D,2976839,49.0
46,8475184,C. Kreider,NYR,L,16789,15,35.3,7.57,2.35,0,...,157.45,Chris Kreider,1991-04-30,L,75,233,87.951666,LW,5833,88.0
76,8476389,V. Trocheck,NYR,C,17927,15,32.6,6.55,2.17,0,...,145.39,Vincent Trocheck,1993-07-11,R,71,187,79.030671,C,2563036,118.0
115,8478048,I. Shesterkin,NYR,G,31035,9,29.6,3.43,3.29,0,...,130.61,Igor Shesterkin,1995-12-30,L,73,197,99.573586,G,3151297,158.0
37,8479323,A. Fox,NYR,D,12639,10,26.4,7.52,2.64,5,...,163.68,Adam Fox,1998-02-17,R,71,185,99.503715,D,4197146,208.0
162,8476979,E. Gustafsson,NYR,D,17041,15,26.3,5.56,1.75,0,...,117.25,Erik Gustafsson,1992-03-14,L,73,189,18.010161,D,3891922,209.0
175,8471734,J. Quick,NYR,G,20028,6,26.2,4.71,4.37,0,...,115.61,Jonathan Quick,1986-01-21,L,73,215,18.732539,G,3634,211.0
189,8476459,M. Zibanejad,NYR,C,17704,15,25.2,5.12,1.68,0,...,112.56,Mika Zibanejad,1993-04-18,R,74,201,96.959286,C,2562637,227.0
227,8482109,A. Lafrenière,NYR,L,14948,15,23.2,5.59,1.55,0,...,103.85,Alexis Lafrenière,2001-10-11,L,73,194,21.359879,LW,4697382,267.0


In [161]:
last_five_df_team
hurricanes = last_five_df_team.loc[last_five_df_team['team'] == 'CAR'].sort_values('gameDate')
summary_statistics(hurricanes)

,playerId,name,team,position,toi,gamesPlayed,goals,assists,specialTeams,shots,hits,blockedShots,powerPlayToi,shorthandedToi,plusMinus,fantasyPoints,secondaryPosition,tertiaryPosition,FPP60,FPPG
12,8478427,S. Aho,CAR,C,6017,5,3,4,2,11,1,1,961,488,0,12.7,C,F,7.60,2.54
5,8476869,B. Skjei,CAR,D,6380,5,1,3,0,12,4,8,45,617,1,10.6,D,D,5.98,2.12
0,8470613,B. Burns,CAR,D,6267,5,2,2,0,8,0,6,550,580,4,9.8,D,D,5.63,1.96
18,8482093,S. Jarvis,CAR,C,5851,5,2,1,1,9,6,2,969,521,0,8.0,C,F,4.92,1.60
8,8476958,J. Slavin,CAR,D,6293,5,0,3,0,11,1,7,25,728,4,7.7,D,D,4.40,1.54
6,8476882,T. Teravainen,CAR,L,5128,5,1,2,2,8,0,3,993,476,0,7.3,W,F,5.12,1.46
14,8480039,M. Necas,CAR,C,5439,5,1,1,1,9,7,3,980,12,-3,6.6,C,F,4.37,1.32
9,8477488,B. Pesce,CAR,D,4988,4,0,0,0,5,1,9,45,421,0,5.1,D,D,3.68,1.27
10,8477950,T. DeAngelo,CAR,D,3762,4,1,1,1,5,0,2,771,0,-3,5.0,D,D,4.78,1.25
4,8476474,S. Noesen,CAR,R,3254,5,2,0,0,7,7,0,542,42,2,5.4,W,F,5.97,1.08
